# Libraries

In [ ]:
import numpy as np
import sklearn.datasets as datasets
import time
import copy
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from tqdm.notebook import tqdm, tnrange

In [ ]:
from DME.Simulation import train

# Helper functions

In [ ]:
def linear_func(n_samples, random_state=1):
    np.random.seed(random_state)
    X = np.random.uniform(size=(n_samples, 30))
    F = 1 + np.sum(X, axis=1)

    return X, F

In [ ]:
def random_binary_generator(n_samples, n_groups, n_obs_per_group, n_features=5, random_state=1):
    np.random.seed(random_state)
    X = np.zeros((n_samples, n_features), dtype=int)
    for i in range(n_features):
        arr = np.random.randint(0, 2, n_groups)
        X[:, i] = np.repeat(arr, n_obs_per_group)
    F = 5*X[:, 0]*X[:, 1] + 3*np.sqrt(2*X[:, 2]+X[:, 3]*X[:, 4]+1)

    return X, F

In [ ]:
def friedman_modified(n_samples, n_features=25, random_state=1):
    np.random.seed(random_state)
    X = np.random.uniform(size=(n_samples, n_features))
    F = 10*np.sin(np.pi*X[:, 0]*X[:, 1]) + 20*(X[:, 2]-0.5)**2 + 10*(X[:, 3]*X[:, 4]) + 3*np.cos(np.pi*(X[:, 5]/X[:, 6])) + (X[:, 7]+X[:, 8]+X[:, 9])**2 + np.sqrt(1+3*X[:, 10]+X[:, 11]) + 30*np.abs(X[:, 12]-X[:, 13]) + X[:, 14]

    return X, F

In [ ]:
def non_linear(n_samples, n_groups, n_obs_per_group, random_state=1):
    np.random.seed(random_state)
    X1, F1 = friedman_modified(n_samples=n_samples, n_features=25, random_state=random_state)
    X2, F2 = random_binary_generator(n_samples, n_groups, n_obs_per_group, random_state=random_state)
    X = np.column_stack((X1, X2))
    F = F1 + F2

    return X, F

In [ ]:
def exponential_kernel(x1, x2, sigma2, rho):
    '''
    Computes the exponential kernel matrix between two vectors.
    '''
    x1 = np.expand_dims(x1, axis=1)
    x2 = np.expand_dims(x2, axis=1)

    return sigma2 * np.exp(-scipy.spatial.distance.cdist(x1, x2, 'sqeuclidean') / rho)

In [ ]:
def generate_data(n, m, p, func, shared_gp=True, random_state=1):
    # Features
    if func == 'non_linear':
        X, F = non_linear(n_samples=n, n_groups=m, n_obs_per_group=p, random_state=random_state)
    elif func == 'linear_func':
        X, F = linear_func(n_samples=n, random_state=random_state)
    else:
        raise Exception('function is invalid') 

    # Create groups
    group = np.arange(n) # Variable that stores group IDs
    for i in range(m):
        group[i*p:(i+1)*p] = i

    # Incidence matrix relating grouped random effects to observations
    Z1 = np.zeros((n, m))
    for i in range(m):
        Z1[np.where(group==i), i] = 1

    # Simulate random (sorted) observation times for each group
    rng = np.random.default_rng(seed=random_state)
    times_arrays = [np.sort(rng.choice(1000, size=p, replace=False, shuffle=False)) for i in range(m)]
    times = np.concatenate(times_arrays)

    # Simulate GPs
    GP_list = []
    np.random.seed(random_state)
    if shared_gp:
        # Simulate GPs with same parameters
        sigma2_2 = 1 ** 2  # Marginal variance of GP
        rho = 0.1  # GP Range parameter
        for arr in times_arrays:
            K = exponential_kernel(arr, arr, sigma2_2, rho)
            g = np.random.multivariate_normal(mean=np.zeros(p), cov=K)
            GP_list.append(g)
    else:
        # Simulate GPs with different, random parameters
        for arr in times_arrays:
            sigma2_2 = scipy.stats.invgamma.rvs(1, loc=0, scale=10)
            rho = np.random.uniform(0.01, 1000)
            K = exponential_kernel(arr, arr, sigma2_2, rho)
            g = np.random.multivariate_normal(mean=np.zeros(p), cov=K)
            GP_list.append(g)

    # Simulate outcome variable
    np.random.seed(random_state)
    sigma2 = 0.5 ** 2  # Error variance
    b = np.random.normal(size=m) # Simulate random effect intercept
    G = np.dot(Z1, b) + np.concatenate(GP_list) # Combine random effect intercept with GP
    epsilon = np.sqrt(sigma2) * np.random.normal(size=n) # Simulate error term
    y = F + G + epsilon

    # Create dataframes
    feature_names = ['feature_' + str(i+1) for i in range(30)]
    data = pd.concat([pd.DataFrame(F, columns=['F']),
                   pd.DataFrame(group, columns=['group']),
                   pd.DataFrame(times, columns=['times']),
                   pd.DataFrame(X,columns=feature_names),
                   pd.DataFrame(y, columns=['y'])],
                   axis=1)

    return data

In [ ]:
def generate_datasets(n, m, p, n_datasets, n_valid, func, shared_gp=True, random_state=1):
    datasets = {}
    datasets['data'] = []
    datasets['dataframes'] = []
    datasets['n_samples_chosen_per_group'] = []

    # Create training and test sets
    for i in range(n_datasets):
        data = generate_data(n, m, p, func, shared_gp, random_state=i+random_state)
        df_new, df_test = train_test_split_grouped_extrapolation(data, data['group'], test_size=0.2, random_state=i)
        group_sizes = df_new.groupby(['group']).size().to_numpy()
        n_samples_chosen_per_group = train_test_split_grouped_interpolation(df_new, group_sizes, test_size=0.2, random_state=i)
        datasets['data'].append(data)
        datasets['dataframes'].append([df_new, df_test])
        datasets['n_samples_chosen_per_group'].append(n_samples_chosen_per_group)
    
    # Create validation datasets
    validation_datasets = {}
    validation_datasets['dataframes'] = []
    for i in range(n_valid):
        data = generate_data(n, m, p, func, shared_gp, random_state=i+n_datasets+random_state)
        df_train, df_val = train_test_split_grouped_extrapolation(data, data['group'], test_size=0.2, random_state=i)
        validation_datasets['dataframes'].append([df_train, df_val])

    return datasets, validation_datasets

In [ ]:
def param_search(dataframes, model_config, train_config,
                 random_effects_column_names,
                 group_column_name, y_column_name,
                 n_samples_chosen_per_group_list,
                 model_type='MLP',
                 random_state=1):
    best_val_error_list = []
    for i in tnrange(len(dataframes)):
        best_state, best_val_err = train.train_and_evaluate_model(model_config, train_config, dataframes[i], 
                                                                  random_effects_column_names,
                                                                  group_column_name, y_column_name,
                                                                  n_samples_chosen_per_group_list[i],
                                                                  model_type, random_state=random_state)
        best_val_error_list.append(best_val_err)

    return np.mean(best_val_error_list)

In [ ]:
def test(dataframes, model_config, train_config,
         random_effects_column_names,
         group_column_name, y_column_name,
         n_samples_chosen_per_group_list,
         model_type='MLP',
         random_state=1):
    F_rmse_test_list, rmse_test1_list, rmse_test2_list = [], [], []
    for i in tnrange(len(dataframes)):
        F_rmse_test1, F_rmse_test2, rmse_test1, rmse_test2 = train.train_and_test_model(model_config, train_config, dataframes[i], random_effects_column_names,
                                                                                        group_column_name, y_column_name,
                                                                                        n_samples_chosen_per_group_list[i],
                                                                                        model_type, random_state=random_state)
        F_rmse_test_list.append(F_rmse_test1)
        F_rmse_test_list.append(F_rmse_test2)
        rmse_test1_list.append(rmse_test1)
        rmse_test2_list.append(rmse_test2)
    
    print('Extrapolation')
    print('Mean: ', np.mean(rmse_test1_list))
    print('Std: ', np.std(rmse_test1_list))
    print('\n')
    print('Interpolation')
    print('Mean: ', np.mean(rmse_test2_list))
    print('Std: ', np.std(rmse_test2_list))
    print('\n')
    print('F')
    print('Mean: ', np.mean(F_rmse_test_list))
    print('Std: ', np.std(F_rmse_test_list))
    
    return np.mean(rmse_test1_list), np.std(rmse_test1_list), np.mean(rmse_test2_list), np.std(rmse_test2_list), np.mean(F_rmse_test_list), np.std(F_rmse_test_list)

In [ ]:
def train_test_split_grouped_interpolation(df, group_sizes, test_size=0.2, random_state=1):
    '''
    Train/test split for a dataframe, but test set contains at least one observation from each group in the training set, and contains no unseen groups.
    '''
    assert 0 < test_size < 1, "Test size must be strictly between 0 and 1"
    assert np.sum(group_sizes) == len(df), "Sum of group_sizes must be equal to length of dataframe"
    assert group_sizes.all() > 0, "Group sizes should be non-negative"
    assert len(group_sizes) < len(df), "Number of groups should be less than number of observations"

    np.random.seed(random_state)
    df_len = len(df)
    test_len = int(test_size * df_len)
    no_groups = len(group_sizes)
    group_sizes_new = group_sizes.copy()

    # Pick one observation from all groups
    sample_len = no_groups
    n_samples_chosen_per_group = np.ones_like(group_sizes_new)
    last_idx_arr = np.cumsum(group_sizes_new)-1 # Array of index of the last observation in each group within the overall dataset
    test_idx = [last_idx_arr[i] for i in range(no_groups)]
    group_sizes_new -= 1

    # Keep picking more observations until the required number of test observations has been picked
    while sample_len < test_len:
        group_idx = np.random.randint(no_groups) # Pick a random group
        if group_sizes_new[group_idx] > 1:
            if test_len - sample_len > 1:
                n = np.random.randint(1, min([group_sizes_new[group_idx], test_len-sample_len])) # Pick a random sample of size 1<=n<group_size from the chosen group
            else:
                n = 1
            last_idx = last_idx_arr[group_idx]-n_samples_chosen_per_group[group_idx] # Index of the last observation remaining in each group within the overall dataset
            test_idx += [last_idx-i for i in range(n)]
            n_samples_chosen_per_group[group_idx] += n # Update number of samples chosen from the group
            group_sizes_new[group_idx] -= n # Update current group sizes
            sample_len += n

    return n_samples_chosen_per_group

In [ ]:
def train_test_split_grouped_extrapolation(df, groups, test_size=0.2, random_state=1):
    '''
    Train/test split for a dataframe, but test set only contains only unseen groups.
    ``test_size`` represents the proportion of groups to include in the test split (rounded up).
    '''
    train_idx, test_idx = next(GroupShuffleSplit(test_size=test_size, random_state=random_state).split(df, groups=groups))
    df_train, df_test = df.iloc[train_idx], df.iloc[test_idx]

    return df_train, df_test

# Experiment 1: Non-linear Function with Temporal Shared GP

In [ ]:
# Generate data
n, m = 1000, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='non_linear', random_state=60)

## MLP

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 1000
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 30
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='MLP')

# print(val_result)

In [ ]:
print(val_result)

8.125364029326665


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='MLP')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  8.69843896254876
Std:  0.5163346282303269


Interpolation
Mean:  8.621200413495583
Std:  0.5388578955427302


F
Mean:  8.266261703341788
Std:  0.5524172569316361


## RNN

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 1000
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 10
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='LSTM')

# print(val_result)

In [ ]:
print(val_result)

8.59337014256057


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='LSTM')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  9.714073195560792
Std:  0.9023682014372404


Interpolation
Mean:  9.399518226160001
Std:  0.8335257645642434


F
Mean:  9.267957489747847
Std:  0.8701517040366222


# Experiment 2: Non-linear Function with Temporal Individual GP

In [ ]:
# Generate data
n, m = 1000, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='non_linear', shared_gp=False, random_state=75)

## MLP

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 1000
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 20
train_config['lr'] = 0.001
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='MLP')

# print(val_result)

In [ ]:
print(val_result)

12.425178509878965


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='MLP')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  11.88768003746365
Std:  2.2521189166493945


Interpolation
Mean:  12.484949856350601
Std:  3.5830453385026475


F
Mean:  8.693517078140527
Std:  0.4544660588942596


## RNN

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 500
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 10
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='LSTM')

# print(val_result)

In [ ]:
print(val_result)

10.564992327459205


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='LSTM')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  12.688886473544366
Std:  2.3046064001575917


Interpolation
Mean:  13.028297518766326
Std:  3.4081207707180994


F
Mean:  9.684766844907276
Std:  0.9220526313912548


# Experiment 3: Linear Function with Temporal Shared GP

In [ ]:
# Generate data
n, m = 1000, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='linear_func', random_state=60)

## MLP

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 500
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 25
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='MLP')

# print(val_result)

In [ ]:
print(val_result)

1.9752435575286966


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='MLP')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  1.9597709351591575
Std:  0.23420589319026025


Interpolation
Mean:  1.8765089095858127
Std:  0.18149579233142207


F
Mean:  0.6962999217870462
Std:  0.20036777725100455


## RNN

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 500
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 10
train_config['lr'] = 0.001
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='LSTM')

# print(val_result)

In [ ]:
print(val_result)

1.889531494907223


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='LSTM')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  1.8725721783936264
Std:  0.20763778195107552


Interpolation
Mean:  1.779909557688964
Std:  0.19100050076231667


F
Mean:  0.37814317368476347
Std:  0.19852596624179686


# Experiment 4: Linear Function with Temporal Independent GP

In [ ]:
# Generate data
n, m = 1000, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='linear_func', shared_gp=False, random_state=40)

## MLP

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 500
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 20
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='MLP')
# print(val_result)

In [ ]:
print(val_result)

7.43874505477191


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='MLP')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  6.115662528268964
Std:  1.9171965119023557


Interpolation
Mean:  7.481036898688825
Std:  4.310866029950044


F
Mean:  1.778840731001122
Std:  0.5840231728221752


## RNN

In [ ]:
model_config = {}
model_config['input_dim'] = 30
model_config['hidden_dim'] = 500
model_config['output_dim'] = 1

train_config = {}
train_config['n_epochs'] = 3
train_config['lr'] = 0.01
train_config['n_adapt'] = 2
train_config['inner_lr'] = 0.1
train_config['l2_penalty'] = 0.001

# val_result = param_search(validation_datasets['dataframes'], model_config, train_config,
#                           random_effects_column_names=['times'],
#                           group_column_name='group', y_column_name='y',
#                           n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
#                           model_type='LSTM')

# print(val_result)

In [ ]:
print(val_result)

7.442349868785516


In [ ]:
test1_mean, test1_std, test2_mean, test2_std, F_mean, F_std = test(datasets['dataframes'], model_config, train_config, random_effects_column_names=['times'],
                                                                   group_column_name='group', y_column_name='y',
                                                                   n_samples_chosen_per_group_list=datasets['n_samples_chosen_per_group'],
                                                                   model_type='LSTM')

In [ ]:
print('Extrapolation')
print('Mean: ', test1_mean)
print('Std: ', test1_std)
print('\n')
print('Interpolation')
print('Mean: ', test2_mean)
print('Std: ', test2_std)
print('\n')
print('F')
print('Mean: ', F_mean)
print('Std: ', F_std)

Extrapolation
Mean:  6.344229075067087
Std:  2.02241664432447


Interpolation
Mean:  7.52363933720652
Std:  4.3031119299264775


F
Mean:  2.008135385401061
Std:  1.2037281862600464
